In [1]:
import time, array, random, copy, math
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

In [3]:
from deap import algorithms, base, benchmarks, tools, creator

In [4]:
from p3_main import *

In [8]:
M = 5 
N = 3
Bounds.x_max = 100	# Determine the size of the area in which Nodes will exist.
Bounds.x_min = -100
Bounds.y_max = 100
Bounds.y_min = -100

Window.number_of_aps = M	# Currently must <= 7 (for visuals) due to color implementation.
Window.number_of_iots = N

# objects.py
AP.max_capacity = 1000	# Determine the capacity for APs.

IOT.min_demand = 0		# Determine the demand range for IOTs.
IOT.max_demand = 100


random.seed(100)
w = Window()
wsn = WSN.getInstance()
iot_list = w.iots
ap_list = w.aps
wsn.calc_PR(ap_list,iot_list)
wsn.calc_SNR()
wsn.calc_rate()
print(wsn.get_rates())

[[11027996.53332779 10298614.50853133 11369791.44067577]
 [12737642.68855594  9485587.56744323 10589512.55809971]
 [11461598.18903311 10598795.82805763 10483031.97297249]
 [10356598.74115969 11474511.06007068 10792707.55448681]
 [11211843.62107359 10890932.68801701 10447483.49363062]]


In [9]:
creator.create("FitnessMulti", base.Fitness, weights=(1.0,-1.0))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMulti)

In [11]:
toolbox = base.Toolbox()
    
toolbox.register("attr", create_c_mat, M, N)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_rate_nap)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", mutFlipAssociation, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 1.0))